In [1]:
from rosemary import jpt_in_notebook, jpt_parse_args, jpt_setup; jpt_setup()
if jpt_in_notebook():
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['CUDA_VISIBLE_DEVICES'].split(',')[1] 
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    
!nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


5
Sat Dec  2 23:29:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   29C    P0    53W / 300W |  18007MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A 

In [2]:
import logging
import os
import sys
from dataclasses import dataclass, field, asdict
from typing import Optional
from functools import partial
import pyarrow
import datasets
import numpy as np

import json
import torch
from datasets import load_dataset
from collections import Counter
import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaTokenizer,
    LlamaTokenizerFast,
    CodeLlamaTokenizerFast,
    HfArgumentParser,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    set_seed,
    GPTNeoXTokenizerFast,
    GPT2Tokenizer,
    GPT2TokenizerFast, 
    OPTForCausalLM,
)
from transformers import Trainer
from peft import LoraConfig, TaskType, get_peft_model
from transformers.trainer_utils import get_last_checkpoint
logger = logging.getLogger(__name__)


## jobs submitted in notebook inherits env variables.
cache_dir = '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/cache'
os.environ['WANDB_DIR'] = cache_dir
os.makedirs(os.environ['WANDB_DIR'], exist_ok=True)
os.environ['WANDB_MODE'] = 'offline'
os.environ['WANDB_PROJECT'] = 'mitibm'
##

[2023-12-02 23:29:13,754] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
import sys
from open_instruct.finetune_trainer import ModelArguments, DataTrainingArguments, TrainingArguments
from open_instruct.finetune_trainer import encode_with_prompt_completion_format, encode_with_messages_format


In [4]:
model_name_or_path = 'results/huggyllama:llama-7b_human_mix-trainer_savebystep'
model_name_or_path = '../results/baselines/huggyllama/llama-7b'
model_name_or_path = '../results/baselines/codellama/CodeLlama-7b-hf'
# model_name_or_path = 'mosaicml/mpt-7b'
# model_name_or_path = '../results/baselines/gpt2-medium'
# model_name_or_path = '../results/baselines/mistralai/Mistral-7B-v0.1'
# model_name_or_path = '../results/baselines/EleutherAI/pythia-70m'

# train_file = 'data/processed/all.jsonl'
# subsample_mixture =  {'cot': 976, 'dolly': 15, 'flan_v2': 976, 'oasst1': 34}; subsample_inds_file=None

# train_file = 'data/processed/flan_v2/flan_v2_data.jsonl'
# train_file = 'data/processed/ultrachat/ultrachat200k_train_data.jsonl'
train_file = 'data/processed/starcoder/starcoder_commentinstr_cleaned.jsonl'


# subsample_inds_file = os.path.join(
#     '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/'
#     'note_explore_dpp_llama-7b_flan_v2_subsets_K_cos.pkl'); subsample_mixture = None
# train_file = 'data/processed/ultrachat/ultrachat_data.jsonl'
subsample_inds_file = None; subsample_mixture = None
dataloader_sampler = 'SequentialSampler' # 'RandomSampler'|'SequentialSampler'
dataloader_sampler = None

#     --use_lora \
#     --lora_rank 8 \
#     --lora_alpha 8 \
#     --lora_dropout 0.05 

save_strategy = 'steps'
save_steps = 5
save_total_limit = 1
eval_steps = save_steps
overwrite_output_dir = True # false if want to test trainer resume from dir.

#     --report_to tensorboard wandb \

cmd = f"""
    --model_name_or_path {model_name_or_path} \
    --tokenizer_name {model_name_or_path} \
    --use_fast_tokenizer True \
    --train_file data/processed/dolly/dolly_data.jsonl \
    --max_seq_length 2048 \
    --train_file {train_file} \
    --do_train \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 128 \
    --gradient_checkpointing \
    --learning_rate 2e-5 \
    --lr_scheduler_type linear \
    --warmup_ratio 0.03 \
    --weight_decay 0. \
    
    --evaluation_strategy=steps \
    --eval_steps={eval_steps} \
    --report_to none \
    --logging_strategy=steps \
    --logging_first_step \
    --logging_steps=1 \
    --save_strategy={save_strategy} \
    --save_steps={save_steps} \
    --save_total_limit={save_total_limit} \
    
    --num_train_epochs 2 \
    --fp16 \
    --torch_dtype float32 \
    --dataloader_num_workers 8 \
    --output_dir "results/jpt_{':'.join(model_name_or_path.split('/')[-2:])}_humanmix" \
    --overwrite_output_dir {overwrite_output_dir} \
    {'--subsample_mixture="'+str(subsample_mixture).replace(': ', ':').replace(', ', ',')+'"'
        if subsample_mixture else ''} \
    {'--subsample_inds_file='+subsample_inds_file if subsample_inds_file else ''} \
    {'--dataloader_sampler '+str(dataloader_sampler) if dataloader_sampler else ''} \
"""
#    --subsample_mixture='{{"flan_v2":100000}}'

#      --subsample_mixture '{{"baize": 8333, "code_alpaca": 8333, "cot": 8333, "dolly": 8333, "flan_v2": 8333, "gpt4_alpaca": 8333, "oasst1": 8333, "self_instruct": 8333, "sharegpt": 8333, "stanford_alpaca": 8333, "super_ni": 8333, "unnatural_instructions": 8333}}'

import shlex
args = shlex.split(cmd)


parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)
# wpq: convert str to dict
if data_args.subsample_mixture is not None:
    import json
    import re
    data_args.subsample_mixture = re.compile('(?<!\\\\)\'').sub('\"', data_args.subsample_mixture)
    data_args.subsample_mixture = json.loads(data_args.subsample_mixture)
    print('subsample mixture:')
    print(data_args.subsample_mixture)
if data_args.subsample_mixture is not None and data_args.subsample_inds_file is not None:
    raise ValueError('Either use mixture proportion or exact subset indices, but not both.')
if data_args.subsample_inds_file is not None:
    if 'flan_v2' not in data_args.train_file:
        raise ValueError('subset indices only support flan_v2 for now.')

model_args, data_args, training_args

(ModelArguments(model_name_or_path='../results/baselines/codellama/CodeLlama-7b-hf', config_name=None, tokenizer_name='../results/baselines/codellama/CodeLlama-7b-hf', cache_dir=None, use_fast_tokenizer=True, model_revision='main', use_auth_token=False, torch_dtype='float32', use_lora=False, lora_rank=64, lora_alpha=16, lora_dropout=0.1, load_in_8bit=False),
 DataTrainingArguments(dataset_name=None, dataset_config_name=None, train_file='data/processed/starcoder/starcoder_commentinstr_cleaned.jsonl', max_train_samples=None, streaming=False, overwrite_cache=False, preprocessing_num_workers=16, max_seq_length=2048, subsample_mixture=None, subsample_inds_file=None),
 TrainingArguments(output_dir='results/jpt_codellama:CodeLlama-7b-hf_humanmix', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, 

In [5]:

# wpq: save args to a json file
with training_args.main_process_first(local=False, desc=f"Saving args to `{training_args.output_dir+'.args.json'}`"):
    args_dict = {
        'model_args': asdict(model_args),
        'data_args': asdict(data_args),
        'training_args': asdict(training_args),
    }
    args_dict_path = training_args.output_dir.strip('/')+'.args.json'
    with open(args_dict_path, 'w') as f:
        json.dump(args_dict, f)
    print(f'Saving args dict to {args_dict_path}')
    

Saving args dict to results/jpt_codellama:CodeLlama-7b-hf_humanmix.args.json


In [6]:
# with training_args.main_process_first(local=False, desc=f"setup wandb init"):
#     if 'wandb' in training_args.report_to:
#         wandb_init_kwargs_path = os.path.join(training_args.output_dir, 'wandb_init_kwargs.json')
#         os.makedirs(training_args.output_dir, exist_ok=True)
#         if not os.path.isfile(wandb_init_kwargs_path):
#             import wandb
#             wandb_init_kwargs = {}
#             wandb_init_kwargs['project'] = os.environ['WANDB_PROJECT']
#             wandb_init_kwargs['name'] = training_args.run_name
#             wandb_init_kwargs['resume'] = 'allow' # resume if `run_id` identical to previous, otherwise start new run
#             wandb_init_kwargs['id'] = wandb.sdk.lib.runid.generate_id()
#             wandb_init_kwargs['mode'] = 'offline'
#             with open(wandb_init_kwargs_path, 'w') as f:
#                 json.dump(wandb_init_kwargs, f, indent=4)
#         else:
#             with open(wandb_init_kwargs_path, 'r') as f:
#                 wandb_init_kwargs = json.load(f)
# wandb_init_kwargs
# import wandb
# wandb.init(**wandb_init_kwargs)

In [6]:

# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        # wpq: since I do write `.json` file to output_dir, raise if there are more files.
        if len(os.listdir(training_args.output_dir))>1:
            raise ValueError(
                f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                "Use --overwrite_output_dir to overcome."
            )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )
        
last_checkpoint

In [7]:

# Set seed before initializing model.
set_seed(training_args.seed)

if data_args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset(
        data_args.dataset_name,
        data_args.dataset_config_name,
        cache_dir=model_args.cache_dir,
        use_auth_token=True if model_args.use_auth_token else None,
        streaming=data_args.streaming,
    )
else:
    data_files = {}
    dataset_args = {}
    if data_args.train_file is not None:
        data_files["train"] = data_args.train_file
    if 'ultrachat' in data_args.train_file:
        data_files['test'] = (
            '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/'
            'data/processed/ultrachat/ultrachat200k_test_data.jsonl')
    raw_datasets = load_dataset(
        "json",
        data_files=data_files,
        cache_dir=model_args.cache_dir,
        use_auth_token=True if model_args.use_auth_token else None,
        **dataset_args,
    )
    if 'ultrachat' in data_args.train_file:
        raw_datasets['test'] = raw_datasets['test'].select(range(1000))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/json/default-609281aa71c1b60c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [58]:

config_kwargs = {
    "cache_dir": model_args.cache_dir,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
    "trust_remote_code": True if 'mpt' in model_args.model_name_or_path else False,
}
if model_args.config_name:
    config = AutoConfig.from_pretrained(model_args.config_name, **config_kwargs)
elif model_args.model_name_or_path:
    config = AutoConfig.from_pretrained(model_args.model_name_or_path, **config_kwargs)
    # wpq: add support for mpt models.
    if 'mpt' in model_args.model_name_or_path:
        config.attn_config['attn_impl'] = 'triton'
        config.init_device = 'cuda' # For fast initialization directly on GPU!
else:
    raise ValueError(
        "You are instantiating a new config instance from scratch. This is not supported by this finetuning script."
    )

tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer if 'pythia' not in model_args.model_name_or_path else True,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
}
if model_args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
elif model_args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this finetuning script."
    )
    
config

LlamaConfig {
  "_name_or_path": "../results/baselines/codellama/CodeLlama-7b-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 16384,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.35.0.dev0",
  "use_cache": true,
  "vocab_size": 32016
}

In [59]:
if model_args.model_name_or_path:
    torch_dtype = (
        model_args.torch_dtype
        if model_args.torch_dtype in ["auto", None]
        else getattr(torch, model_args.torch_dtype)
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
        torch_dtype=torch_dtype,
        # wpq: 8bit training
        load_in_8bit=model_args.load_in_8bit,
        trust_remote_code=bool('mpt' in model_args.model_name_or_path),
    )
else:
    logger.warning("No pretrained model_name_or_path is given. Training new model from scratch.")
    model = AutoModelForCausalLM.from_config(config)
    n_params = sum({p.data_ptr(): p.numel() for p in model.parameters()}.values())
    logger.info(f"Training new model from scratch - Total size={n_params/2**20:.2f}M params")
    

# wpq: `use_cache=True` is incompatible with gradient checkpointing
model.config.use_cache = True if not training_args.gradient_checkpointing else False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [76]:

# no default pad token for llama!
# here we add all special tokens again, because the default ones are not in the special_tokens_map 
if isinstance(tokenizer, (LlamaTokenizer, LlamaTokenizerFast, CodeLlamaTokenizerFast)):
    from transformers import AddedToken
    num_added_tokens = tokenizer.add_special_tokens({
        "bos_token": AddedToken("<s>", normalized=False, special=True),
        "eos_token": AddedToken("</s>", normalized=False, special=True),
        "unk_token": AddedToken("<unk>", normalized=False, special=True),
        "pad_token": AddedToken("<pad>", normalized=False, special=True),
    })
    ## wpq: for `huggyllama`/`NousResearch/Llama-2-7b-hf`, `LlamaTokenizerFast` tokenizer config not properly implemented and cannot tokenize special tokens like eos_token corretly. Need the followign workaround. More details: https://github.com/huggingface/transformers/issues/23833
    if isinstance(tokenizer, LlamaTokenizerFast):
        print('here')
        from secrets import token_hex
        tmp_tok_path = f'/tmp/wpq_tok_{token_hex(16)}'
        tokenizer.save_pretrained(tmp_tok_path)
        tokenizer = AutoTokenizer.from_pretrained(tmp_tok_path, **tokenizer_kwargs)

    for s, s_tokenized in [
        ("Hi<s>Hey</s>sir<unk>what<pad><pad>", 
        ['▁Hi', '<s>', '▁Hey', '</s>', '▁sir', '<unk>', '▁what', '<pad>', '<pad>']),
    ]:
        assert(tokenizer.tokenize(s, add_special_tokens=False)==s_tokenized)
elif isinstance(tokenizer, GPTNeoXTokenizerFast):
    num_added_tokens = tokenizer.add_special_tokens({
        "pad_token": "<pad>",
    })
    assert num_added_tokens == 1, "GPTNeoXTokenizer should only add one special token - the pad_token."
elif isinstance(tokenizer, (GPT2Tokenizer, GPT2TokenizerFast)) and isinstance(model, OPTForCausalLM):
    num_added_tokens = tokenizer.add_special_tokens({'unk_token': '<unk>'})
## wpq: add support for gpt2 tokenizer.
elif isinstance(tokenizer, (GPT2Tokenizer, GPT2TokenizerFast)):
    num_added_tokens = tokenizer.add_special_tokens({
        "pad_token": "<pad>",
    })
    assert num_added_tokens == 1, "GPT2Tokenizer should only add one special token - the pad_token."

# resize embeddings if needed (e.g. for LlamaTokenizer)
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))

    
# wpq: use int8 training
# wpq: put this after resize embedding!
if model_args.load_in_8bit:
    from peft import prepare_model_for_int8_training
    model = prepare_model_for_int8_training(model)

In [82]:

# wpq: add peft to finetune_trainer.py
if model_args.use_lora:
    logger.info("Initializing LORA model...")
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM, 
        inference_mode=False, 
        bias='none',
        r=model_args.lora_rank, 
        lora_alpha=model_args.lora_alpha, 
        lora_dropout=model_args.lora_dropout,
        target_modules=['q_proj','k_proj','v_proj','o_proj'],
    )
    # wpq: the following fixes `element 0 of tensors does not require grad and does not have a grad_fn` 
    # https://github.com/huggingface/peft/issues/137
    # https://github.com/huggingface/peft/issues/522
    if hasattr(training_args, 'gradient_checkpointing'):
        if training_args.gradient_checkpointing:
            model.enable_input_require_grads()

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

In [84]:

# Preprocessing the datasets.
if "prompt" in raw_datasets["train"].column_names and "completion" in raw_datasets["train"].column_names:
    encode_function = partial(
        encode_with_prompt_completion_format,
        tokenizer=tokenizer,
        max_seq_length=data_args.max_seq_length,
    )
elif "messages" in raw_datasets["train"].column_names:
    encode_function = partial(
        encode_with_messages_format,
        tokenizer=tokenizer,
        max_seq_length=data_args.max_seq_length,
    )
else:
    raise ValueError("You need to have either 'prompt'&'completion' or 'messages' in your column names.")

raw_datasets['train'] = raw_datasets['train'].select(range(1000))
    

# To speed up this part, we use multiprocessing.
with training_args.main_process_first(local=False, desc="Processing instruction data"):
    if not data_args.streaming:
        lm_datasets = raw_datasets.map(
            encode_function,
            batched=False,
            num_proc=data_args.preprocessing_num_workers,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Tokenizing and reformatting instruction data",
        )
    else:
        lm_datasets = raw_datasets.map(
            encode_function,
            batched=False,
        )
    lm_datasets.set_format(type="pt")

if training_args.do_train:
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = lm_datasets["train"]
    ## wpq: subsample `dataset` according to `data_args.subsample_mixture`.
    # assumes dataset in `train_file` ordered, 
    # e.g., ['cot', 'cot', 'flan_v2', 'flan_v2', ...]
    # note `counts.items()` is ordered as well!
    if data_args.subsample_mixture is not None:
        counts = Counter(train_dataset['dataset'])
        inds = []
        cum = 0
        for k, N in counts.items():
            n = int(data_args.subsample_mixture.get(k, 0))
            replace = True if n>N else False
            replace = True # always sample with replacement, for fairer comparison.
            inds += list(np.random.choice(N, size=n, replace=replace) + cum)
            cum += N
            print(k, N, n, len(inds))
        train_dataset = train_dataset.select(inds)


    if data_args.subsample_inds_file is not None:
        with open(data_args.subsample_inds_file, 'rb') as f:
            inds = pickle.load(f)['K']
        logger.info(f'Using subsample_inds_file: {data_args.subsample_inds_file}')
        logger.info(f'subsample_inds_file has {len(inds)} indices.')
        train_dataset = train_dataset.select(inds)

    ## 
    if data_args.max_train_samples is not None:
        max_train_samples = min(len(train_dataset), data_args.max_train_samples)
        train_dataset = train_dataset.select(range(max_train_samples))


Tokenizing and reformatting instruction data (num_proc=16):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [85]:
from open_instruct.finetune_trainer import MyTrainer

# initalize a trainer
# here we use a custom trainer that moves the model to CPU when saving the checkpoint in FSDP mode
# we can switch to the default trainer after moving to deepspeed (let's don't change too much for now)
trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=lm_datasets['test'] if (training_args.do_eval and 'test' in lm_datasets) else None,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
)

In [25]:
checkpoint = None
if training_args.resume_from_checkpoint is not None:
    checkpoint = training_args.resume_from_checkpoint
elif last_checkpoint is not None:
    checkpoint = last_checkpoint
train_result = trainer.train(resume_from_checkpoint=checkpoint)

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenize

Step,Training Loss,Validation Loss
5,0.000000,nan


You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenize

In [88]:
!cd .. && python open_instruct/reformat_datasets.py --raw_data_dir data/raw_train --output_dir data/processed --dataset starcoder


Processing starcoder data...


In [90]:
from note_pruning_analysis import get_dataset

dss = get_dataset('starcoder_simple')
dsr = get_dataset('starcoder_role')


Found cached dataset json (/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/starcoder/json/default-81508a354854d02d/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/starcoder/json/default-122cd053399382de/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


In [98]:
# instrs = list(zip(
#     [x[0]['content'] for x in dss[:100]['messages']],
#     [x[0]['content'] for x in dsr[:100]['messages']],
# ))

In [102]:
for i in range(20):
    print('\nsimple:\n')
    print('user:\n')
    messages = dss[i]['messages']
    print(messages[0]['content'])
    print('assistant:\n')
    print(messages[1]['content'])
    print('\nrole:\n')
    print('user:\n')
    messages = dsr[i]['messages']
    print(messages[0]['content'])
    print('assistant:\n')
#     print(messages[1]['content'])
    
    print('\n\n\n\n\n')


simple:

user:

Task: Implement a convolutional neural network (CNN) for image classification using Keras. The model should use CIFAR-10 dataset, which consists of 50,000 training images and 10,000 testing images. The aim is to classify the images into 10 different classes. The layers of the model should include Conv2D, Activation, MaxPooling2D, Dropout, and Dense. The activation function should be relu in the first two Conv2D layers and softmax in the last Dense layer. The optimizer should be RMSprop with a learning rate of 0.0001 and a decay of 1e-6. The loss function should be categorical crossentropy. The model should train for 100 epochs and should be evaluated on the test set.


assistant:

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

batch_size = 32
num_classes = 10
epochs = 100

# The data, spli

In [ ]:

trainer.save_model()  # Saves the tokenizer too for easy upload

metrics = train_result.metrics

max_train_samples = (
    data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

In [ ]:
wandb.finish()